## Particle within radius or overlap calculations

In [1]:
#dataframes
import pandas as pd
import h5py

#speedup
import multiprocessing as mp

#suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.TimeSeries = pd.Series 

#math
import numpy as np
import math as m
from scipy.spatial.distance import cdist


#plots
import pylab as plt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar
import seaborn as sns

#misc
import time
np.set_printoptions(suppress=True)

In [51]:
dir_save = '../../data/Sprints/AN/'
dir ='~/Documents/MyFiles/DataAnalysis/data/Sprints/HighRes/'
df = pd.read_hdf(dir+'NotWindy.h5')

In [4]:
# df = df.drop(columns=['relative_perpendicular_comp','relative_parallel_comp',
#                       'distance_from_source_',
#                       'nearest_from_streakline_', 'corrected_u', 'corrected_v'
#                       ])

In [7]:
pi = np.pi/180
x = np.abs(df.S2*np.sin(((df.D+13)*pi)))
y = np.abs(df.S2*np.cos(((df.D+13)*pi)))

x = x*np.sign(df.U)
y = y*np.sign(df.V)

df['corrected_u'] = x
df['corrected_v'] = y

In [8]:
df.reset_index(inplace=True, drop=True) 

### Calculate Expected encounters

In [9]:
# df.drop(df.loc[(df['time'] > 6200)].index,inplace=True) 

In [10]:
# df = df.drop(columns=['time'])
dt=df.master_time[1]-df.master_time[0]

time = []
time.append(0)
for i in range(1,len(df)):
    time.append(time[i-1]+dt)
    
df['time'] = time

In [6]:
# df.insert(1,'sync_time',sync_time)

# storing the odor location in a numpy array to avoid reading pandas
source = np.array([[0,0]])
odor_position = np.array([[df.xsrc[i],df.ysrc[i]] for i in range (len(df.xsrc))]) 

distance_from_source = np.array([cdist(odor_position,source)]).flatten()

In [12]:
df['distance_from_source'] = distance_from_source

### Relative Motion, and Distance Calculation

In [19]:
def relative_components(streakline_slope,sensor_dir,wind_dir,wind_speed,sensor_speed):
   
    ## perpendicular component
    wind_perpendicular_comp = np.sin(wrapped_angular_diff(streakline_slope,wind_dir))*wind_speed
    sensor_perpendicular_comp = np.sin(wrapped_angular_diff(streakline_slope,sensor_dir))*sensor_speed
    perp.append(wind_perpendicular_comp-sensor_perpendicular_comp)
    
    ## parallel component
    wind_parallel_comp = np.cos(wrapped_angular_diff(streakline_slope,wind_dir))*wind_speed
    sensor_parallel_comp = np.cos(wrapped_angular_diff(streakline_slope,sensor_dir))*sensor_speed
    paral.append(wind_parallel_comp-sensor_parallel_comp)
    

In [20]:
def wrapped_angular_diff(a, b):
    return np.arctan2(np.sin(a-b), np.cos(a-b))

def tangent_slope_direction(slope, x): 
    if (x >= 0):
        return slope
    else:
        if (slope > 0):
            return(3.14159265359 - slope)            
        else:
            return(-3.14159265359-(slope))
        
def relative_magnitude_calculation(i):
    wind_speed_magnitude = np.linalg.norm([df.corrected_u[i],df.corrected_v[i]])
    sensor_speed_magnitude = np.linalg.norm([df.gps_linear_x[i],df.gps_linear_y[i]])
    return wind_speed_magnitude, sensor_speed_magnitude

In [21]:
def relative_direction_calculation(streakline, pos, i):
    ## get the tangent from the nearest point
    if(pos == 0):
        x = streakline[pos:(pos+2),0]
        y = streakline[pos:(pos+2),1]
    else:
        x = streakline[(pos-1):(pos+2),0]
        y = streakline[(pos-1):(pos+2),1]
    slope_streakline,intercept_streakline=np.polyfit(x,y,1)
    
    streakline_slope=tangent_slope_direction(np.arctan(slope_streakline),streakline[pos,0] )
    
    ## find direction - source location
    sensor_dir = np.arctan2(df.gps_linear_y[i],df.gps_linear_x[i])
    wind_dir = np.arctan2(df.corrected_v[i],df.corrected_u[i])
    wind_speed, sensor_speed = relative_magnitude_calculation(i)
    
    relative_components(streakline_slope,sensor_dir,wind_dir,wind_speed,sensor_speed)    

In [22]:
def compute_distance(streakline, odor_position,i):
    source = np.array([[0,0]])
    streakline = np.concatenate((streakline,source))
    distance = cdist(odor_position,streakline).flatten()   # cdist compares distance for all the points in both arrays
#     nearest_from_streakline.append(np.min(distance))
    pos = np.where(distance == np.amin(distance))
    relative_direction_calculation(streakline, pos[0][0],i)
    
#     if (len(pos[0]) > 1):
#         length=(np.sqrt(np.sum(np.diff(np.array(streakline[int(pos[0][0]):]), axis=0)**2, axis=1)))
#     else:
#         length=(np.sqrt(np.sum(np.diff(np.array(streakline[int(pos[0]):]), axis=0)**2, axis=1)))
        
#     distance_along_streakline.append(np.sum(length))

In [23]:
def find_streakline():
    eastwest = [np.sum(df.corrected_u[j:])*dt for j in range(0,len(df))]
    northsouth = [np.sum(df.corrected_v[j:])*dt for j in range(0,len(df))]
    return eastwest,northsouth

In [24]:
# eastwest,northsouth = find_streakline()

In [26]:
st = pd.read_hdf(dir+'NotWindyStreaklineCorrected.h5')
eastwest = st.eastwest
northsouth= st.northsouth

In [27]:
len(df)

1289622

In [28]:
perp = []
paral = []
# nearest_from_streakline = []
# distance_along_streakline = []

for i in range(len(eastwest)-1, 1000000, -1):
    odor_pos = [odor_position[i]] 
    eastwest = np.resize(np.array([eastwest-df.corrected_u[i]*dt]),(1,i)).flatten() # resize needed to avoid negative data
    northsouth = np.resize(np.array([northsouth-df.corrected_v[i]*dt]),(1,i)).flatten()
    wind_pos = np.vstack([eastwest[-6000:],northsouth[-6000:]]).T   
    compute_distance(wind_pos,odor_pos,i)

In [34]:
for i in range(len(eastwest)-1, 6000, -1):
    odor_pos = [odor_position[i]] 
    eastwest = np.resize(np.array([eastwest-df.corrected_u[i]*dt]),(1,i)).flatten() # resize needed to avoid negative data
    northsouth = np.resize(np.array([northsouth-df.corrected_v[i]*dt]),(1,i)).flatten()
    wind_pos = np.vstack([eastwest[-6000:],northsouth[-6000:]]).T   
    compute_distance(wind_pos,odor_pos,i)

In [42]:
#for the section when only the length of the elements is equal to the window value
for i in range((len(eastwest))-1, -1, -1):
    
    odor_pos = [odor_position[i]]  
    if(i == 0):
        radius = np.zeros(1)
        wind_pos = np.array([[0,0]])
        distance = cdist(odor_pos,wind_pos).flatten()
        paral.append(0)
        perp.append(0)
#         nearest_from_streakline.append(np.min(distance))
        
    else:
        eastwest = np.resize(np.array([eastwest-df.corrected_u[i]*dt]),(1,i)).flatten() # resize needed to avoid negative data
        northsouth = np.resize(np.array([northsouth-df.corrected_v[i]*dt]),(1,i)).flatten()
        wind_pos = np.vstack([eastwest,northsouth]).T   
        compute_distance(wind_pos,odor_pos,i)

In [37]:
## flip containers because above iteration is done in reverse order

# distance_along_streakline = np.array(np.flip(distance_along_streakline)) 
# nearest_from_streakline = np.array(np.flip(nearest_from_streakline)) 

# odor_presence = odor_presence[::-1]

## storing in the dataframe 
# df['distance_along_streakline'] =  distance_along_streakline
# df['nearest_from_streakline_'] = nearest_from_streakline
df['relative_parallel_comp']=paral[::-1]
df['relative_perpendicular_comp']=perp[::-1]

In [38]:
dir ='~/Documents/MyFiles/DataAnalysis/data/Sprints/HighRes/'
df.to_hdf(dir+'NotWindyPP_.h5', key='df', mode='w')

In [44]:
len(paral)

1289622

#### Extract Continuous Streakline

In [7]:
dir ='~/Documents/MyFiles/DataAnalysis/data/Sprints/HighRes/'
df = pd.read_hdf(dir+'Windy.h5')

In [8]:
time_res=2002*5 ## 50 seconds for highres 
range_points = []
start_angle=df.D[0]
range_points.append(0)
i = 0        
while i < (len(df)):
    diff = np.abs(df.D[i] - start_angle)
    if diff >= 60:
        range_points.append(i)
        start_angle=df.D[i+time_res]
        i+=time_res
    else:
        i+=1
        continue

In [9]:
cs=pd.DataFrame()
j = 0
while j < (len(range_points)):
#     print(i)
    if(j < len(range_points)-1):
        cs = cs.append(df[range_points[j]:range_points[j+1]])        
    else:
        cs = cs.append(df[range_points[j]:])
    j+=2
cs.reset_index(inplace=True, drop=True) 


In [10]:
dt=df.time[1]-df.time[0]
time = []
time.append(0)
for i in range(1,len(cs)):
    time.append(time[i-1]+dt)

cs = cs.drop(columns=['time'])
cs['time']=time

In [11]:
len(cs)

347059

In [12]:
cs.to_hdf(dir+'ContinuousWindy.h5', key='cs', mode='w')